# Ejercicio: Contar palabras en reseñas con UDF en PySpark


Este notebook carga el archivo `ejercicio_reviews_simple.csv`, define una **UDF en PySpark** para contar
las palabras de cada reseña y genera una nueva columna `num_palabras` con ese resultado.


## 1) Configuración inicial

In [1]:

import os, sys

# Asegurar mismo Python para driver y workers (Windows)
os.environ["PYSPARK_PYTHON"] = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable

from pyspark.sql import SparkSession

spark = (SparkSession.builder
         .appName("Ejercicio_UDF_Reviews")
         .config("spark.python.use.daemon", "false")
         .config("spark.python.worker.reuse", "false")
         .config("spark.executorEnv.PYSPARK_PYTHON", sys.executable)
         .getOrCreate())

spark


## 2) Cargar el archivo CSV

In [2]:

# Ajusta la ruta al archivo CSV si es necesario
csv_path = "ejercicio_reviews_simple.csv"

df = (spark.read
      .option("header", True)
      .option("inferSchema", True)
      .csv(csv_path))

df.show(truncate=False)
df.printSchema()


+---+-----------------------------------------+
|id |texto                                    |
+---+-----------------------------------------+
|1  |Muy buen servicio y atención rápida.     |
|2  |Producto defectuoso, no funcionó.        |
|3  |Excelente calidad, volvería a comprar.   |
|4  |Entrega lenta, pero producto aceptable.  |
|5  |El empaque llegó dañado.                 |
|6  |Me encantó, lo recomiendo totalmente.    |
|7  |No cumplió con mis expectativas.         |
|8  |Buen diseño y fácil de usar.             |
|9  |Precio demasiado alto para la calidad.   |
|10 |Superó mis expectativas, cinco estrellas.|
+---+-----------------------------------------+

root
 |-- id: integer (nullable = true)
 |-- texto: string (nullable = true)



## 3) Definir la UDF para contar palabras

In [3]:

from pyspark.sql.functions import udf, col
from pyspark.sql.types import IntegerType

def contar_palabras(s):
    try:
        if not s:
            return 0
        s = str(s)
        return sum(1 for t in s.split() if t.strip())
    except Exception:
        return 0

udf_contar_palabras = udf(contar_palabras, IntegerType())


## 4) Crear columna `num_palabras` y mostrar resultado

In [4]:

df2 = df.withColumn("num_palabras", udf_contar_palabras(col("texto")))
df2.show(truncate=False)


+---+-----------------------------------------+------------+
|id |texto                                    |num_palabras|
+---+-----------------------------------------+------------+
|1  |Muy buen servicio y atención rápida.     |6           |
|2  |Producto defectuoso, no funcionó.        |4           |
|3  |Excelente calidad, volvería a comprar.   |5           |
|4  |Entrega lenta, pero producto aceptable.  |5           |
|5  |El empaque llegó dañado.                 |4           |
|6  |Me encantó, lo recomiendo totalmente.    |5           |
|7  |No cumplió con mis expectativas.         |5           |
|8  |Buen diseño y fácil de usar.             |6           |
|9  |Precio demasiado alto para la calidad.   |6           |
|10 |Superó mis expectativas, cinco estrellas.|5           |
+---+-----------------------------------------+------------+

